In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
DATA_PATH = "/home/dude/Desktop/data/hotel_cancelation/hotel_booking.csv"

In [3]:
df = pd.read_csv(DATA_PATH)

In [4]:
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,email,phone-number,credit_card
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,Transient,0.0,0,0,Check-Out,2015-07-01,Ernest Barnes,Ernest.Barnes31@outlook.com,669-792-1661,************4322
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,Transient,0.0,0,0,Check-Out,2015-07-01,Andrea Baker,Andrea_Baker94@aol.com,858-637-6955,************9157
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,2015-07-02,Rebecca Parker,Rebecca_Parker@comcast.net,652-885-2745,************3734
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,2015-07-02,Laura Murray,Laura_M@gmail.com,364-656-8427,************5677
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,Transient,98.0,0,1,Check-Out,2015-07-03,Linda Hines,LHines@verizon.com,713-226-5883,************5498


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 36 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [6]:
df['children'].fillna(0, inplace=True)

/tmp/ipykernel_9993/289084130.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['children'].fillna(0, inplace=True)


In [7]:
df['country'].fillna("NOG", inplace=True) # Not Given

/tmp/ipykernel_9993/1526170687.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['country'].fillna("NOG", inplace=True) # Not Given


In [8]:
df.drop('agent', inplace=True, axis=1)

In [9]:
df.drop('company', inplace=True, axis=1)

In [10]:
df['hotel'] = df['hotel'].apply(lambda x: 0 if x == 'City Hotel' else 1)

In [11]:
df['arrival_date_month'] = pd.to_datetime(df['arrival_date_month'], format='%B').dt.month

In [12]:
df.drop(['credit_card', 'email', 'phone-number', 'name'], axis=1, inplace=True)

In [13]:
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])

In [14]:
df['reservation_day'] = df['reservation_status_date'].apply(lambda x: x.day)
df['reservation_month'] = df['reservation_status_date'].apply(lambda x: x.month)
df['reservation_year'] = df['reservation_status_date'].apply(lambda x: x.year)

In [15]:
df.drop('reservation_status_date', axis=1, inplace=True)

In [16]:
dummy_columns = ['meal', 'market_segment', 'distribution_channel', 'reserved_room_type', 'assigned_room_type', 
                 'deposit_type', 'customer_type', 'reservation_status']
for col in dummy_columns: 
    dummies = pd.get_dummies(df[col], dtype=int)
    df = pd.concat([df, dummies], axis=1)
    df.drop(col, axis=1, inplace=True)


In [20]:
countries = pd.get_dummies(df['country'], dtype=int)
countries = pd.concat([df['is_canceled'], countries])

In [25]:
df.drop('country', axis=1, inplace=True)

In [34]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline

In [31]:
X = df.drop('is_canceled', axis=1)
y = df['is_canceled']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [33]:
from sklearn.linear_model import LogisticRegression

In [35]:
log_reg_pipeline = make_pipeline(
    StandardScaler(), LogisticRegression(C=1)
)

In [36]:
log_reg_pipeline.fit(X_train, y_train)
y_pred = log_reg_pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18720
           1       1.00      1.00      1.00     11128

    accuracy                           1.00     29848
   macro avg       1.00      1.00      1.00     29848
weighted avg       1.00      1.00      1.00     29848



In [37]:
print(confusion_matrix(y_test, y_pred))

[[18720     0]
 [    0 11128]]


In [39]:
from joblib import dump
print(type(log_reg_pipeline))

<class 'sklearn.pipeline.Pipeline'>


In [40]:
dump(log_reg_pipeline, "hotel_cancelation_clf.joblib")

['hotel_cancelation_clf.joblib']